In [ ]:
!pip install --quiet openai transformers sentencepiece

In [ ]:

import os
import sys
import json
try:

    from openai import OpenAI
except Exception:
    OpenAI = None


try:
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
except Exception:
    AutoTokenizer = None
    AutoModelForSeq2SeqLM = None


try:
    from getpass import getpass
except Exception:
    getpass = input




def use_openai_generate(topic, client=None, model="gpt-4o-mini"):
    """Generate 3 sub-questions and a summary using OpenAI Chat API.
    Returns tuple: (subquestions_text, summary_text)
    """
    if client is None:
        raise RuntimeError("OpenAI client not provided")


    sub_qs_prompt = (
        f"You are a research assistant. Generate exactly 3 detailed, focused research sub-questions "
        f"based on this topic: \"{topic}\". Number them 1., 2., 3. Keep each sub-question to one sentence."
    )

    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": sub_qs_prompt}],
        temperature=0.2,
        max_tokens=300,
    )

    sub_qs_text = resp.choices[0].message.content.strip()


    summary_prompt = (
        "Summarize the following three sub-questions into one concise paragraph that describes the research focus: \n"
        + sub_qs_text
    )

    resp2 = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": summary_prompt}],
        temperature=0.2,
        max_tokens=200,
    )

    summary_text = resp2.choices[0].message.content.strip()
    return sub_qs_text, summary_text


def hf_fallback_generate(topic, hf_model_name="google/flan-t5-base"):
    """Simple fallback using FLAN-T5 to perform the two steps.
    This is NOT as reliable as OpenAI chat but works offline if the model is available locally.
    Returns tuple: (subquestions_text, summary_text)
    """
    if AutoTokenizer is None or AutoModelForSeq2SeqLM is None:
        raise RuntimeError("Transformers not available. Install with: pip install transformers sentencepiece")

    tokenizer = AutoTokenizer.from_pretrained(hf_model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(hf_model_name)

    prompt1 = (
        f"Generate exactly 3 detailed research sub-questions based on the topic: {topic}. Number them 1.,2.,3."
    )
    inputs = tokenizer(prompt1, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=256)
    sub_qs_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    prompt2 = "Summarize the following sub-questions into a single concise paragraph: \n" + sub_qs_text
    inputs2 = tokenizer(prompt2, return_tensors="pt")
    outputs2 = model.generate(**inputs2, max_new_tokens=128)
    summary_text = tokenizer.decode(outputs2[0], skip_special_tokens=True)

    return sub_qs_text, summary_text




def main():
    print(" Mini AI Research Assistant ")
    topic = input("Enter your research topic: ").strip()
    if not topic:
        print("No topic entered. Exiting.")
        return


    api_key = os.environ.get("OPENAI_API_KEY")
    client = None
    if api_key is None and OpenAI is not None:

        print("OpenAI API key not found in environment.")
        k = getpass("Paste your OpenAI API key (will be hidden): ")
        if k:
            api_key = k.strip()
            os.environ["OPENAI_API_KEY"] = api_key

    if api_key and OpenAI is not None:
        try:
            client = OpenAI(api_key=api_key)
            sub_qs, summary = use_openai_generate(topic, client=client)
            print("\nGenerated Sub-questions:\n")
            print(sub_qs)
            print("\nSummary:\n")
            print(summary)
            return
        except Exception as e:
            print("OpenAI call failed:", str(e))
            print("Falling back to Hugging Face (if available)...")


    try:
        sub_qs, summary = hf_fallback_generate(topic)
        print("\nGenerated Sub-questions (HF fallback):\n")
        print(sub_qs)
        print("\nSummary:\n")
        print(summary)
        return
    except Exception as e:
        print("Fallback failed:", str(e))
        print("Unable to complete the task. Make sure you installed required packages and provided an OpenAI API key.")


if __name__ == "__main__":
    main()


 Mini AI Research Assistant 
Enter your research topic: Impact of Internet on People

Generated Sub-questions:

1. How has the proliferation of social media platforms influenced interpersonal communication and relationships among different age groups?  
2. In what ways has access to online information and resources affected individuals' decision-making processes regarding health and wellness?  
3. What are the psychological effects of internet addiction on users, particularly in relation to their productivity and mental health?

Summary:

This research focuses on the multifaceted impact of digital technology on human behavior and relationships, examining how the rise of social media platforms has transformed interpersonal communication across various age groups, influenced decision-making in health and wellness through increased access to online information, and affected mental health and productivity due to internet addiction.
